# Libraries

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sc
import datetime as dt
import yfinance as yfin
import plotly.graph_objects as go
from pandas_datareader import data as pdr
from scipy.stats import norm, t

yfin.pdr_override() # pandas_datareader Yahoo Finance has some problems from 2023 so we need this override

# Efficient Frontier

## Efficient Frontier - Code

In [17]:
# Import Data

def getData(stocks, start, end):
# Remember to check the tickers before calculating the function
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']                            # We are only interested in Close prices
    returns = stockData.pct_change()                          # Furthermore, we are only interested in percentage changes over time
    meanReturns = returns.mean()                              # Calculating the meanReturns
    covMatrix = returns.cov()                                 # Calculating the covMatrix
    return meanReturns , covMatrix

In [18]:
# Finding the performance (returns,std) of a Portfolio (weights)

def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252                 # We are multiplying for 252 (number of trading days in a year)
    std = np.sqrt(np.dot(weights.T,np.dot(covMatrix, weights)))*np.sqrt(252)
    return returns, std

In [19]:
# Calculating the Maximum Sharpe Ratio Portfolio - SharpeRatio = (return - riskfree Rate) / std

def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0):        # Getting negative Sharpe Ratio to minimize
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd
def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet=(0,1)): # Minimize the negative SR, by altering the weights of the portfolio
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)                         # Given data
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})        # Sum of % weights should be equal to 1
    bound = constraintSet                                                 # Weights can range in (0,1)
    bounds = tuple(bound for asset in range(numAssets))                   # We have numAssets weights to range in (0,1)
    result = sc.minimize(negativeSR, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints) #SLSQP Sequential Least Squares Programming
    return result

In [20]:
# Calculating the Minimum Volatility Portfolio

def portfolioVariance(weights, meanReturns, covMatrix):                   # Minimizing the std equals minimizing the Variance
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [21]:
# Isolating portfolioReturn
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]

# Calculating the Minimum Varaiance for a given Target return
def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)): # For each returnTarget, we want to optimise the portfolio for min variance
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget}, # We fixed the return
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

In [22]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    # Input: mean, cov matrix, and other financial information
    # Output: Max SR Portfolio , Min Volatility Portfolio, Efficient frontier Portfolios Performance
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns,4), round(maxSR_std,4)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]

    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns,4), round(minVol_std,4)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]
    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])
    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns

In [23]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)): # Return a graph ploting the min vol Portfolio, max sr Portfolio and the efficient frontier
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)
    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std*100],
        y=[maxSR_returns*100],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )
    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std*100],
        y=[minVol_returns*100],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )
    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )
    data = [MaxSharpeRatio, MinVol, EF_curve]
    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 1.25, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)

    fig = go.Figure(data=data, layout=layout)
    return fig.show()

## Efficient Frontier - Example

In [24]:
# Stocks Example - Choose your favorite stocks - Use Yahoo Finance Tickers
stocks = ['AAPL','GOLD','PFE']

# Dates Example - 1 year back from now
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

# getData Example
meanReturns , covMatrix = getData(stocks, startDate, endDate)

# Graph
EF_graph(meanReturns, covMatrix)

[*********************100%%**********************]  3 of 3 completed


#Var and CVar

## Var and CVar - Code

In [25]:
# Import Data

def getData1(stocks, start, end):
# Remember to check the tickers before calculating the function
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']                            # We are only interested in Close prices
    returns = stockData.pct_change()                          # Furthermore, we are only interested in percentage changes over time
    meanReturns = returns.mean()                              # Calculating the meanReturns
    covMatrix = returns.cov()                                 # Calculating the covMatrix
    return returns, meanReturns , covMatrix

In [26]:
# Finding the performance (returns,std) of a Portfolio (weights)

def portfolioPerformance1(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time                # We will input the Time we want to go back
    std = np.sqrt(np.dot(weights.T,np.dot(covMatrix, weights)))*np.sqrt(Time)
    return returns, std

## Historical Var & CVar - Code

In [27]:
def historicalVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the percentile of the distribution at the given alpha confidence level
    """
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    # A passed user-defined-function will be passed a Series for evaluation.
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalVaR, alpha = alpha)
    else:
        raise TypeError("Expected returns to be dataframe or series")

In [28]:
def historicalCVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the CVaR for dataframe / series
    """
    if isinstance(returns, pd.Series):
        belowVaR = returns <= historicalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()
    # A passed user-defined-function will be passed a Series for evaluation.
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalCVaR, alpha=alpha)
    else:
        raise TypeError("Expected returns to be dataframe or series")

## Parametric Var & CVar - Code

In [29]:
def var_parametric(portofolioReturns, portfolioStd, distribution='normal', alpha=5, dof=6):
    # because the distribution is symmetric
    if distribution == 'normal':
        VaR = norm.ppf(1-alpha/100)*portfolioStd - portofolioReturns                             # De-Normalization of a Normal Distribution -> Var = -(q_alpha(5)*std + mean) = -(-q_alpha(95)*std + mean) = q_alpha(95)*std - mean
    elif distribution == 't-distribution':
        nu = dof
        VaR = np.sqrt((nu-2)/nu) * t.ppf(1-alpha/100, nu) * portfolioStd - portofolioReturns      # De-Normalization of a t-Distribution
    else:
        raise TypeError("Expected distribution type 'normal'/'t-distribution'")
    return VaR

def cvar_parametric(portofolioReturns, portfolioStd, distribution='normal', alpha=5, dof=6):      # dof = degree of freedom in t-distribution
    if distribution == 'normal':
        CVaR = (alpha/100)**-1 * norm.pdf(norm.ppf(alpha/100))*portfolioStd - portofolioReturns
    elif distribution == 't-distribution':
        nu = dof
        xanu = t.ppf(alpha/100, nu)
        CVaR = -1/(alpha/100) * (1-nu)**(-1) * (nu-2+xanu**2) * t.pdf(xanu, nu) * portfolioStd - portofolioReturns
    else:
        raise TypeError("Expected distribution type 'normal'/'t-distribution'")
    return CVaR

## Monte Carlo Var & CVar - Code

In [30]:
# Monte Carlo Simulation
def MonteCarlo(mc_sims, T, weights, meanReturns, covMatrix, initialPortfolio = 10000):
  meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
  meanM = meanM.T
  portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)
  for m in range(0, mc_sims):
      # MC loops
      Z = np.random.normal(size=(T, len(weights)))
      L = np.linalg.cholesky(covMatrix)
      dailyReturns = meanM + np.inner(L, Z)
      portfolio_sims[:,m] = np.cumprod(np.inner(weights, dailyReturns.T)+1)*initialPortfolio
  return portfolio_sims

In [31]:
def mcVaR(returns, alpha=5):
    """ Input: pandas series of returns
        Output: percentile on return distribution to a given confidence level alpha
    """
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    else:
        raise TypeError("Expected a pandas data series.")
def mcCVaR(returns, alpha=5):
    """ Input: pandas series of returns
        Output: CVaR or Expected Shortfall to a given confidence level alpha
    """
    if isinstance(returns, pd.Series):
        belowVaR = returns <= mcVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()
    else:
        raise TypeError("Expected a pandas data series.")

## Var and Cvar - Example

In [32]:
# Stocks - Choose your favourite stocks - Remember to use Yahoo Finance tickers
stocks = ['TSLA','AAPL','MSFT', 'NVDA']
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=800)

# Getting Data
returns, meanReturns , covMatrix = getData1(stocks, startDate, endDate)
returns = returns.dropna()

# Random Weights
weights = np.random.random(len(returns.columns)) # We are using Random Weights but we can fix them
weights /= np.sum(weights)
returns['portfolio'] = returns.dot(weights) # Adding a columns to the Dataframe to contain the portfolio return on a certain date

# Example 100 Days in the future
mc_sims = 400 # Number of simulations
Time = 100 # Timeframe in days
hVaR = -historicalVaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
hCVaR = -historicalCVaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
pRet, pStd = portfolioPerformance1(weights, meanReturns, covMatrix, Time)
normVaR = var_parametric(pRet, pStd)
normCVaR = cvar_parametric(pRet, pStd)
tVaR = var_parametric(pRet, pStd, distribution='t-distribution')
tCVaR = cvar_parametric(pRet, pStd, distribution='t-distribution')

# Monte-Carlo
InitialInvestment = 10000
portfolio_sims= MonteCarlo(mc_sims, Time, weights, meanReturns, covMatrix)
portResults = pd.Series(portfolio_sims[-1,:])
VaR = InitialInvestment - mcVaR(portResults, alpha=5)
CVaR = InitialInvestment - mcCVaR(portResults, alpha=5)

# Comparison of each VaR & CVaR methods

print("\nVaR:")
print(' historical VaR 95th CI   :      ', round(InitialInvestment*hVaR,2))
print(" Normal VaR 95th CI       :      ", round(InitialInvestment*normVaR,2))
print(" t-dist VaR 95th CI       :      ", round(InitialInvestment*tVaR,2))
print(" MC VaR  95th CI          :      ", round(VaR,2))
print("\nCVaR:")
print(' historical CVaR 95th CI  :      ', round(InitialInvestment*hCVaR,2))
print(" Normal CVaR 95th CI      :      ", round(InitialInvestment*normCVaR,2))
print(" t-dist CVaR 95th CI      :      ", round(InitialInvestment*tCVaR,2))
print(" MC CVaR 95th CI          :      ", round(CVaR,2))

[*********************100%%**********************]  4 of 4 completed

VaR:
 historical VaR 95th CI   :       3773.11
 Normal VaR 95th CI       :       2711.09
 t-dist VaR 95th CI       :       2584.75
 MC VaR  95th CI          :       2448.03

CVaR:
 historical CVaR 95th CI  :       4697.8
 Normal CVaR 95th CI      :       3617.3
 t-dist CVaR 95th CI      :       3819.97
 MC CVaR 95th CI          :       3308.36
